# E-commerce Data Analysis - Comprehensive EDA

This notebook provides a comprehensive exploratory data analysis of e-commerce sales data, focusing on business metrics and performance indicators. The analysis is structured to be configurable for different date ranges and reusable for future datasets.

## Table of Contents
1. [Introduction & Business Objectives](#introduction)
2. [Data Loading & Configuration](#data-loading)
3. [Data Preparation & Transformation](#data-preparation)
4. [Business Metrics Calculation](#business-metrics)
   - 4.1 [Revenue Analysis](#revenue-analysis)
   - 4.2 [Product Performance](#product-performance)
   - 4.3 [Geographic Analysis](#geographic-analysis)
   - 4.4 [Customer Experience Analysis](#customer-experience)
5. [Summary of Observations](#summary)

## Data Dictionary
Key business terms and column definitions:
- **Revenue**: Total price value of delivered orders
- **AOV**: Average Order Value - mean total price per order
- **MoM Growth**: Month-over-Month growth percentage
- **Delivery Speed**: Days between order placement and delivery
- **Customer Satisfaction**: Average review score (1-5 scale)
- **order_status**: Current status (delivered, canceled, processing, shipped, pending, returned)
- **price**: Item price excluding shipping costs
- **freight_value**: Shipping cost per item

## 1. Introduction & Business Objectives {#introduction}

### Business Context
This analysis examines e-commerce performance metrics to understand:
- Revenue trends and growth patterns
- Product category performance
- Geographic sales distribution
- Customer satisfaction and delivery performance

### Analysis Goals
- Compare performance between different time periods
- Identify top-performing product categories and regions
- Assess customer experience through delivery speed and satisfaction metrics
- Provide actionable insights for business decision-making

## 2. Data Loading & Configuration {#data-loading}

Configure analysis parameters and load required datasets using our custom data loading module.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_loader import load_all_datasets, create_analysis_ready_dataset, get_data_dictionary, validate_data_quality
from business_metrics import (
    calculate_revenue_metrics, calculate_monthly_growth_trend, calculate_order_metrics,
    calculate_product_category_performance, calculate_geographic_performance,
    calculate_customer_experience_metrics, calculate_order_status_distribution,
    generate_monthly_revenue_data, filter_data_by_date_range
)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# Analysis Configuration
# Modify these parameters to analyze different time periods

DATA_DIRECTORY = 'ecommerce_data'
TARGET_YEAR = 2023  # Primary year for analysis
COMPARISON_YEAR = 2022  # Year to compare against
ANALYSIS_MONTHS = None  # Set to (start_month, end_month) to filter specific months, e.g., (1, 6) for Jan-Jun

# Color scheme for consistent business-oriented visualizations
BUSINESS_COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72', 
    'success': '#F18F01',
    'warning': '#C73E1D',
    'neutral': '#6C757D'
}

print(f"Analysis Configuration:")
print(f"Target Year: {TARGET_YEAR}")
print(f"Comparison Year: {COMPARISON_YEAR}")
print(f"Month Filter: {ANALYSIS_MONTHS if ANALYSIS_MONTHS else 'Full Year'}")
print(f"Data Directory: {DATA_DIRECTORY}")

In [ ]:
# Load and validate datasets
print("Loading e-commerce datasets...")

# Load all datasets
datasets = load_all_datasets(DATA_DIRECTORY)

# Create analysis-ready dataset
sales_data, supporting_datasets = create_analysis_ready_dataset(
    DATA_DIRECTORY, 
    filter_delivered=True, 
    include_reviews=True
)

print(f"Datasets loaded successfully:")
for name, df in datasets.items():
    print(f"  {name}: {len(df):,} rows, {len(df.columns)} columns")

print(f"\nMain sales dataset: {len(sales_data):,} rows (delivered orders only)")

In [ ]:
# Data quality validation
print("Performing data quality checks...")
quality_report = validate_data_quality(datasets)

# Display key quality metrics
for dataset_name, metrics in quality_report.items():
    print(f"\n{dataset_name.upper()} Dataset:")
    print(f"  Total rows: {metrics['total_rows']:,}")
    print(f"  Duplicate rows: {metrics['duplicate_rows']:,}")
    print(f"  Memory usage: {metrics['memory_usage_mb']:.2f} MB")
    
    # Show top missing value columns
    missing_vals = {k: v for k, v in metrics['missing_values'].items() if v > 0}
    if missing_vals:
        print(f"  Missing values: {dict(list(missing_vals.items())[:3])}")

## 3. Data Preparation & Transformation {#data-preparation}

Apply date range filtering and prepare datasets for analysis based on configuration parameters.

In [ ]:
# Apply date range filtering based on configuration
if ANALYSIS_MONTHS:
    start_month, end_month = ANALYSIS_MONTHS
    print(f"Filtering data for months {start_month} to {end_month}")
    
    sales_data_filtered = filter_data_by_date_range(
        sales_data, 
        'order_purchase_timestamp',
        min(TARGET_YEAR, COMPARISON_YEAR),
        max(TARGET_YEAR, COMPARISON_YEAR),
        start_month,
        end_month
    )
else:
    print("Using full year data for analysis")
    sales_data_filtered = sales_data.copy()

print(f"Filtered dataset: {len(sales_data_filtered):,} rows")
print(f"Date range: {sales_data_filtered['order_purchase_timestamp'].min()} to {sales_data_filtered['order_purchase_timestamp'].max()}")
print(f"Years available: {sorted(sales_data_filtered['year'].unique())}")
print(f"Target year data: {len(sales_data_filtered[sales_data_filtered['year'] == TARGET_YEAR]):,} rows")
print(f"Comparison year data: {len(sales_data_filtered[sales_data_filtered['year'] == COMPARISON_YEAR]):,} rows")

In [ ]:
# Display sample of prepared data
print("Sample of analysis-ready dataset:")
display(sales_data_filtered.head())

print("\nDataset structure:")
print(sales_data_filtered.info())

## 4. Business Metrics Calculation {#business-metrics}

Calculate comprehensive business metrics using our custom business metrics module.

### 4.1 Revenue Analysis {#revenue-analysis}

Analyze revenue performance, growth trends, and order patterns.

In [ ]:
# Calculate revenue metrics
revenue_metrics = calculate_revenue_metrics(sales_data_filtered, TARGET_YEAR, COMPARISON_YEAR)

print(f"REVENUE ANALYSIS ({TARGET_YEAR} vs {COMPARISON_YEAR})")
print("=" * 50)
print(f"Revenue {TARGET_YEAR}: ${revenue_metrics['target_year_revenue']:,.2f}")
print(f"Revenue {COMPARISON_YEAR}: ${revenue_metrics['comparison_year_revenue']:,.2f}")
print(f"Revenue Growth: {revenue_metrics['revenue_growth_percent']:.2f}%")

growth_direction = "increase" if revenue_metrics['revenue_growth_percent'] > 0 else "decrease"
print(f"\nRevenue {growth_direction} of {abs(revenue_metrics['revenue_growth_percent']):.2f}% year-over-year")

In [ ]:
# Calculate monthly growth trend
monthly_growth = calculate_monthly_growth_trend(sales_data_filtered, TARGET_YEAR)

print(f"\nMONTHLY GROWTH TREND - {TARGET_YEAR}")
print("=" * 40)
print(f"Average Monthly Growth: {monthly_growth['average_monthly_growth_percent']:.2f}%")
print(f"Months Analyzed: {monthly_growth['months_analyzed']}")

# Show month-by-month growth
monthly_data = pd.DataFrame.from_dict(monthly_growth['monthly_growth_series'], orient='index', columns=['Growth'])
monthly_data['Growth_Percent'] = monthly_data['Growth'] * 100
monthly_data.dropna(inplace=True)

print("\nMonth-over-Month Growth by Month:")
for month, growth in monthly_data['Growth_Percent'].items():
    print(f"  Month {month}: {growth:+.2f}%")

In [ ]:
# Calculate order metrics
order_metrics = calculate_order_metrics(sales_data_filtered, TARGET_YEAR, COMPARISON_YEAR)

print(f"\nORDER PERFORMANCE ({TARGET_YEAR} vs {COMPARISON_YEAR})")
print("=" * 50)
print(f"Total Orders {TARGET_YEAR}: {order_metrics['target_year_orders']:,}")
print(f"Total Orders {COMPARISON_YEAR}: {order_metrics['comparison_year_orders']:,}")
print(f"Order Count Growth: {order_metrics['order_count_growth_percent']:.2f}%")

print(f"\nAverage Order Value {TARGET_YEAR}: ${order_metrics['target_year_avg_order_value']:.2f}")
print(f"Average Order Value {COMPARISON_YEAR}: ${order_metrics['comparison_year_avg_order_value']:.2f}")
print(f"AOV Growth: {order_metrics['avg_order_value_growth_percent']:.2f}%")

In [ ]:
# Visualize monthly revenue trend
monthly_revenue_data = generate_monthly_revenue_data(sales_data_filtered, TARGET_YEAR)

plt.figure(figsize=(12, 6))
plt.plot(monthly_revenue_data['month'], monthly_revenue_data['price'], 
         marker='o', linewidth=2.5, markersize=8, color=BUSINESS_COLORS['primary'])
plt.title(f'Monthly Revenue Trend - {TARGET_YEAR}', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13))

# Format y-axis as currency
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add value labels on data points
for i, row in monthly_revenue_data.iterrows():
    plt.annotate(f'${row["price"]:,.0f}', 
                xy=(row['month'], row['price']), 
                xytext=(0, 10), textcoords='offset points',
                ha='center', fontsize=9, alpha=0.8)

plt.tight_layout()
plt.show()

### 4.2 Product Performance {#product-performance}

Analyze performance by product category to identify top-performing segments.

In [ ]:
# Calculate product category performance
category_performance = calculate_product_category_performance(
    sales_data_filtered, 
    supporting_datasets['products'], 
    TARGET_YEAR
)

print(f"PRODUCT CATEGORY PERFORMANCE - {TARGET_YEAR}")
print("=" * 50)
print(f"Total Categories: {len(category_performance)}")
print(f"Top 5 Categories by Revenue:")

for i, row in category_performance.head().iterrows():
    print(f"  {i+1}. {row['product_category_name']}: ${row['total_revenue']:,.2f} ({row['order_count']} orders)")

print(f"\nDetailed Category Performance:")
display(category_performance.head(10))

In [ ]:
# Visualize product category performance
top_categories = category_performance.head(10)

plt.figure(figsize=(14, 8))
bars = plt.bar(range(len(top_categories)), top_categories['total_revenue'], 
               color=BUSINESS_COLORS['primary'], alpha=0.8)

plt.title(f'Top 10 Product Categories by Revenue - {TARGET_YEAR}', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Product Category', fontsize=12)
plt.ylabel('Total Revenue ($)', fontsize=12)
plt.xticks(range(len(top_categories)), 
           [cat.replace('_', ' ').title() for cat in top_categories['product_category_name']], 
           rotation=45, ha='right')

# Format y-axis as currency
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'${height:,.0f}',
             ha='center', va='bottom', fontsize=9, rotation=0)

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 4.3 Geographic Analysis {#geographic-analysis}

Examine sales performance across different states and regions.

In [ ]:
# Calculate geographic performance
geographic_performance = calculate_geographic_performance(
    sales_data_filtered,
    datasets['orders'],
    supporting_datasets['customers'],
    TARGET_YEAR
)

print(f"GEOGRAPHIC PERFORMANCE - {TARGET_YEAR}")
print("=" * 40)
print(f"Total States: {len(geographic_performance)}")
print(f"Top 10 States by Revenue:")

for i, row in geographic_performance.head(10).iterrows():
    print(f"  {i+1}. {row['customer_state']}: ${row['total_revenue']:,.2f} ({row['order_count']} orders)")

print(f"\nDetailed Geographic Performance:")
display(geographic_performance.head(15))

In [ ]:
# Create choropleth map for geographic visualization
fig = px.choropleth(
    geographic_performance,
    locations='customer_state',
    color='total_revenue',
    locationmode='USA-states',
    scope='usa',
    title=f'Revenue by State - {TARGET_YEAR}',
    color_continuous_scale='Blues',
    labels={'total_revenue': 'Revenue ($)'}
)

fig.update_layout(
    title_font_size=16,
    title_x=0.5,
    width=900,
    height=600
)

fig.show()

# Alternative bar chart for top states
top_states = geographic_performance.head(15)

plt.figure(figsize=(12, 8))
bars = plt.barh(range(len(top_states)), top_states['total_revenue'], 
                color=BUSINESS_COLORS['secondary'], alpha=0.8)

plt.title(f'Top 15 States by Revenue - {TARGET_YEAR}', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Total Revenue ($)', fontsize=12)
plt.ylabel('State', fontsize=12)
plt.yticks(range(len(top_states)), top_states['customer_state'])

# Format x-axis as currency
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add value labels
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width + width*0.01, bar.get_y() + bar.get_height()/2.,
             f'${width:,.0f}',
             ha='left', va='center', fontsize=9)

plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 4.4 Customer Experience Analysis {#customer-experience}

Analyze customer satisfaction through delivery performance and review scores.

In [ ]:
# Calculate customer experience metrics
customer_experience = calculate_customer_experience_metrics(
    sales_data_filtered,
    supporting_datasets['reviews'],
    TARGET_YEAR
)

print(f"CUSTOMER EXPERIENCE METRICS - {TARGET_YEAR}")
print("=" * 50)
print(f"Average Delivery Time: {customer_experience['average_delivery_days']:.1f} days")
print(f"Average Review Score: {customer_experience['average_review_score']:.2f} / 5.0")
print(f"Orders with Reviews: {customer_experience['total_orders_with_reviews']:,}")

print(f"\nSatisfaction by Delivery Speed:")
for speed_category, avg_score in customer_experience['delivery_satisfaction_by_speed'].items():
    print(f"  {speed_category}: {avg_score:.2f} / 5.0")

In [ ]:
# Calculate order status distribution
order_status_dist = calculate_order_status_distribution(datasets['orders'], TARGET_YEAR)

print(f"\nORDER STATUS DISTRIBUTION - {TARGET_YEAR}")
print("=" * 40)
print(f"Total Orders: {order_status_dist['total_orders']:,}")

for status, percentage in order_status_dist['status_distribution_percent'].items():
    print(f"  {status.title()}: {percentage:.1f}%")

In [ ]:
# Visualize review score distribution
if 'review_score' in sales_data_filtered.columns:
    target_year_data = sales_data_filtered[sales_data_filtered['year'] == TARGET_YEAR]
    review_dist = target_year_data['review_score'].value_counts(normalize=True).sort_index()
    
    plt.figure(figsize=(10, 6))
    bars = plt.barh(review_dist.index, review_dist.values, 
                    color=BUSINESS_COLORS['success'], alpha=0.8)
    
    plt.title(f'Review Score Distribution - {TARGET_YEAR}', 
              fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Proportion of Reviews', fontsize=12)
    plt.ylabel('Review Score', fontsize=12)
    plt.yticks(review_dist.index)
    
    # Add percentage labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + width*0.01, bar.get_y() + bar.get_height()/2.,
                 f'{width:.1%}',
                 ha='left', va='center', fontsize=10)
    
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize delivery satisfaction by speed
delivery_satisfaction = pd.DataFrame.from_dict(
    customer_experience['delivery_satisfaction_by_speed'], 
    orient='index', 
    columns=['Avg_Review_Score']
).reset_index()
delivery_satisfaction.columns = ['Delivery_Speed', 'Avg_Review_Score']

plt.figure(figsize=(10, 6))
bars = plt.bar(delivery_satisfaction['Delivery_Speed'], 
               delivery_satisfaction['Avg_Review_Score'],
               color=BUSINESS_COLORS['warning'], alpha=0.8)

plt.title(f'Average Review Score by Delivery Speed - {TARGET_YEAR}', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Delivery Speed Category', fontsize=12)
plt.ylabel('Average Review Score', fontsize=12)
plt.ylim(0, 5)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 5. Summary of Observations {#summary}

Based on the comprehensive analysis of the e-commerce data, here are the key findings and business insights:

In [ ]:
# Generate automated summary insights
print(f"EXECUTIVE SUMMARY - E-COMMERCE PERFORMANCE ANALYSIS")
print("=" * 60)

# Revenue Performance
print(f"\n1. REVENUE PERFORMANCE ({TARGET_YEAR} vs {COMPARISON_YEAR})")
print(f"   Total Revenue {TARGET_YEAR}: ${revenue_metrics['target_year_revenue']:,.2f}")
revenue_change = "increased" if revenue_metrics['revenue_growth_percent'] > 0 else "decreased"
print(f"   Revenue {revenue_change} by {abs(revenue_metrics['revenue_growth_percent']):.1f}% year-over-year")

# Order Performance
print(f"\n2. ORDER PERFORMANCE")
print(f"   Total Orders {TARGET_YEAR}: {order_metrics['target_year_orders']:,}")
print(f"   Average Order Value: ${order_metrics['target_year_avg_order_value']:.2f}")
order_change = "increased" if order_metrics['order_count_growth_percent'] > 0 else "decreased"
print(f"   Order volume {order_change} by {abs(order_metrics['order_count_growth_percent']):.1f}%")

# Product Performance
print(f"\n3. PRODUCT CATEGORY INSIGHTS")
top_category = category_performance.iloc[0]
print(f"   Top Category: {top_category['product_category_name'].replace('_', ' ').title()}")
print(f"   Top Category Revenue: ${top_category['total_revenue']:,.2f}")
print(f"   Total Categories: {len(category_performance)}")

# Geographic Performance
print(f"\n4. GEOGRAPHIC DISTRIBUTION")
top_state = geographic_performance.iloc[0]
print(f"   Top State: {top_state['customer_state']}")
print(f"   Top State Revenue: ${top_state['total_revenue']:,.2f}")
print(f"   States Served: {len(geographic_performance)}")

# Customer Experience
print(f"\n5. CUSTOMER EXPERIENCE")
print(f"   Average Delivery Time: {customer_experience['average_delivery_days']:.1f} days")
print(f"   Average Review Score: {customer_experience['average_review_score']:.2f} / 5.0")
delivered_pct = order_status_dist['status_distribution_percent'].get('delivered', 0)
print(f"   Successful Delivery Rate: {delivered_pct:.1f}%")

# Monthly Trend
print(f"\n6. GROWTH TRENDS")
avg_monthly_growth = monthly_growth['average_monthly_growth_percent']
trend_direction = "positive" if avg_monthly_growth > 0 else "negative"
print(f"   Average Monthly Growth: {avg_monthly_growth:.2f}% ({trend_direction} trend)")

print(f"\n" + "=" * 60)
print(f"Analysis completed for {TARGET_YEAR} with {COMPARISON_YEAR} comparison")
print(f"Configuration: {len(sales_data_filtered):,} delivered orders analyzed")
if ANALYSIS_MONTHS:
    print(f"Time Period: Months {ANALYSIS_MONTHS[0]}-{ANALYSIS_MONTHS[1]}")
else:
    print(f"Time Period: Full year analysis")

### Key Business Recommendations

Based on the analysis, here are actionable insights for business improvement:

**Revenue Optimization:**
- Focus on the top-performing product categories for inventory and marketing investment
- Investigate reasons for revenue changes to develop targeted strategies

**Geographic Expansion:**
- Consider expanding marketing efforts in high-performing states
- Analyze logistics and delivery infrastructure in underperforming regions

**Customer Experience Enhancement:**
- Optimize delivery speed, especially for categories with longer delivery times
- Monitor review scores and address factors contributing to lower ratings
- Implement strategies to improve the overall delivery success rate

**Operational Efficiency:**
- Analyze seasonal trends to optimize inventory management
- Focus on reducing order cancellations and processing delays

---

*This analysis can be easily reconfigured for different time periods by modifying the configuration parameters at the beginning of the notebook.*